# 分散表現の作成

- gensimを用いた分散表現の作成

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import re
import spacy
import gensim

# matplotlib: 日本語フォントの設定
from matplotlib import rcParams
rcParams['font.family'] = 'sans-serif'
rcParams['font.sans-serif'] = ['Hiragino Maru Gothic Pro', 'Yu Gothic', 'Meirio', 
                               'Takao', 'IPAexGothic', 'IPAPGothic', 'Noto Sans CJK JP']

# Spacyの日本語モデルをロード
nlp = spacy.load('ja_core_news_lg')

# フィードデータの読み込み
feeds = pd.read_csv('data/output_jp.csv')

# title から　'\u300012/4 10:35更新'　の部分を削除
feeds['title'] = feeds['title'].map(lambda x: x[:x.rfind('\u3000')] if x[-2:] == '更新' else x)

# text と summary を結合
feeds['text'] = feeds['title'].str.cat(feeds['summary'], sep='。', na_rep='')

# 不要になった列を削除した処理用の DataFrame
df = feeds.drop(['title', 'summary'], axis=1)

# 確認
df.head()

### 日本語テキストに対する前処理

- 表記の正規化
- トークン化（形態素解析）
- ストップワードの除去
- 見出し語化

In [ ]:
# 不要な単語を除去
# - ストップワード (is_stop)
# - いくつかの品詞
#     AUX: 助動詞
#     PUNCT: 句読点
#     SPACE: 空白文字
#     SYM: 記号
#     X: その他
# - うまく取り除けない単語や文字
stop_pos = ['AUX', 'PUNCT', 'SPACE', 'SYM', 'X']
stop_words = ['.']

def token_to_add(w):
    t = w.text    # 単語
    p = w.pos_    # 品詞
    l = w.lemma_  # 原型

    # ストップワードは None を返す
    if w.is_stop:
        return None
    if p in stop_pos:
        return None
    if l in stop_words:
        return None

    if len(l) == 0:
        return t
    return l

def preprocess(text):
    tokens = []
    
    for w in nlp(text):
        t = token_to_add(w)
        if t is not None:
            tokens.append(t)

    # トークンのリストを返す
    return tokens

### 分散表現の作成

- トークン化したテキストのリストを作成
- CBOW モデルを作成

In [ ]:
# トークン化したテキストのリスト
tokened_text = []

for text in df['text']:
    tokened_text.append(preprocess(text))

# 確認（最初の2行）
tokened_text[:2]

In [ ]:
# デフォルトでは CBOW モデルを作成
model = gensim.models.Word2Vec(tokened_text)

### 類似単語の検索

In [ ]:
model.wv.most_similar('コロナ')

### 単語の類推 (Word Analogy)

In [ ]:
# 日本にとっての首相は、アメリカ（米国）にとっては何か？
# 首相 - 日本 + 米国
model.wv.most_similar(positive=['首相', '米国'], negative=['日本'])